## Install Dependencies

In [ ]:
%pip install kagglehub pandas
%pip install -q transformers peft datasets accelerate bitsandbytes sentencepiece pydantic huggingface_hub xformers
%pip install optuna

#%pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128


## Configurations  

In [33]:
# ==============================
# 🛠 CONFIGURATION
# ==============================

class Config:
    JSON_OUTPUT_DIR = "json_outputs_all_data"
    JSON_OUTPUT_NORMALIZED_DIR = "json_outputs_all_data/normalized"
    JSON_OUTPUT_NORMALIZED_JD = "json_outputs_all_data/normalized/jd"
    JSON_OUTPUT_NORMALIZED_RESUME = "json_outputs_all_data/normalized/resume"
    JSON_OUTPUT_SCORING_DIR = "json_outputs_all_data/scoring"
    JSON_OUTPUT_SCORING_SPLIT_DIR = "json_outputs_all_data/scoring/split"
    JSON_OUTPUT_SCORING_FT_DATA = "json_outputs_all_data/scoring/FT_data"
    JSON_OUTPUT_FINE_TUNE_SCORE = "json_outputs_all_data/fine-tune/scored"
    JSON_OUTPUT_FINE_TUNE_RECORD = "json_outputs_all_data/fine-tune/record"
    JSON_OUTPUT_FINE_TUNE_TEST_DATA = "json_outputs_all_data/fine-tune/test-data"
    JSON_OUTPUT_FINE_TUNE_OUTPUT = "json_outputs_all_data/fine-tune/optuna_output"
    JSON_OUTPUT_FINE_TUNE_MODEL = "json_outputs_all_data/fine-tune/model"

## Login to huggingface

In [34]:
from huggingface_hub import login
import os

# Set your token here securely or prompt for it in Colab
# Recommended: store in Colab secrets or environment variable
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")


if not HF_TOKEN:
    # Prompt for token if not set in environment
    print("🔑 Please enter your Hugging Face token:")
    # For Colab or local prompt input
    HF_TOKEN = input("🔑 Enter your Hugging Face token: ").strip()

login(token=HF_TOKEN)


# Full Fine-Tuning on Lambda with Optuna, LR Scheduler, Early Stopping

### Imports & Configuration

In [9]:
import optuna
import os
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
import torch
from transformers import BitsAndBytesConfig



### Paths & Basic Config

In [4]:
MODEL_NAME = "Qwen/Qwen2-7B-Instruct"


# ✅ Paths
train_path =os.path.join(Config.JSON_OUTPUT_FINE_TUNE_TEST_DATA, "train.jsonl") 
eval_path = os.path.join(Config.JSON_OUTPUT_FINE_TUNE_TEST_DATA, "eval.jsonl") 



### Load Tokenizer & Dataset

In [5]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


data = load_dataset("json", data_files={"train": train_path, "validation": eval_path})


### Tokenization Function

In [6]:
def tokenize(example):
    prompt = f"<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n{example['output']}<|im_end|>"
    tokens = tokenizer(prompt, padding="max_length", truncation=True, max_length=1024)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


tokenized_data = data.map(tokenize, remove_columns=data["train"].column_names)

### Define Optuna Objective Function

In [9]:
def objective(trial):
    from peft import prepare_model_for_kbit_training

    # Hyperparameter suggestions
    learning_rate = trial.suggest_float("learning_rate", 5e-5, 5e-4, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 4)
    lora_r = trial.suggest_categorical("lora_r", [4, 8, 16])
    lora_alpha = trial.suggest_categorical("lora_alpha", [16, 32, 64])

    # Load base model with bitsandbytes quantization (no meta tensor error)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )

    base_model = prepare_model_for_kbit_training(base_model)

    # Apply LoRA
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )
    model = get_peft_model(base_model, lora_config)

    # Output directories for current trial
    output_dir = os.path.join(Config.JSON_OUTPUT_FINE_TUNE_OUTPUT, f"optuna_trial_{trial.number}")
    logging_dir = os.path.join(output_dir, "logs")

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        bf16=True,
        load_best_model_at_end=True,
        report_to="none",
        save_total_limit=1,
        logging_dir=logging_dir,
        logging_steps=10,
    )

    # Prepare trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["validation"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # Training and evaluation
    trainer.train()
    trial.set_user_attr("best_model_path", trainer.state.best_model_checkpoint or output_dir)
    eval_metrics = trainer.evaluate()
    return eval_metrics["eval_loss"]


### Launch Optuna Tuning

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

print("✅ Best hyperparameters:")
print(study.best_params)


[I 2025-06-24 03:06:12,954] A new study created in memory with name: no-name-d8fbb683-ed2c-439c-a685-0b3cfe68b5f2
Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.40s/it]
/tmp/ipykernel_9503/3243740175.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

Epoch,Training Loss,Validation Loss
1,1.937700,1.863273
2,1.669700,1.700749
3,1.683100,1.627911
4,1.671900,1.596733


/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/

[I 2025-06-24 16:58:54,497] Trial 0 finished with value: 1.5967330932617188 and parameters: {'learning_rate': 0.0004189916968040283, 'num_train_epochs': 4, 'lora_r': 4, 'lora_alpha': 32}. Best is trial 0 with value: 1.5967330932617188.
Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.41s/it]
/tmp/ipykernel_9503/3243740175.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_

Epoch,Training Loss,Validation Loss
1,2.006100,1.937912
2,1.730800,1.770430
3,1.772300,1.719054


...

Epoch,Training Loss,Validation Loss
1,1.563300,1.477697
2,1.267400,1.278966


/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[I 2025-06-26 07:28:35,931] Trial 4 finished with value: 1.2789655923843384 and parameters: {'learning_rate': 0.00032850708239078144, 'num_train_epochs': 2, 'lora_r': 16, 'lora_alpha': 64}. Best is trial 4 with value: 1.2789655923843384.
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.11s/it]
/tmp/ipykernel_9503/3243740175.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if us

Epoch,Training Loss,Validation Loss
1,2.177500,2.109524


/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


### Resume Optuna trial

In [19]:
import optuna
import os
import json
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback
#from transformers.utils import logging
from datasets import load_dataset, DatasetDict
from transformers import BitsAndBytesConfig
import torch

# Setup
completed_trials = set()
base_output_dir = Config.JSON_OUTPUT_FINE_TUNE_OUTPUT

# Detect completed trials based on output folders
for entry in os.listdir(base_output_dir):
    if entry.startswith("optuna_trial_"):
        try:
            trial_id = int(entry.replace("optuna_trial_", ""))
            trial_path = os.path.join(base_output_dir, entry)

            # 🔍 Search for checkpoint-*/trainer_state.json
            for subdir in sorted(os.listdir(trial_path), reverse=True):
                if subdir.startswith("checkpoint-"):
                    checkpoint_path = os.path.join(trial_path, subdir, "trainer_state.json")
                    if os.path.exists(checkpoint_path):
                        with open(checkpoint_path) as f:
                            state = json.load(f)
                            if state.get("best_model_checkpoint", None):
                                completed_trials.add(trial_id)
                        break  # Stop after finding the first valid checkpoint
        except ValueError:
            continue

print(f"✅ Detected completed trials: {sorted(completed_trials)}")

# ⛏️ Define objective
def objective(trial):
    trial_number = trial.number
    if trial_number in completed_trials:
        print(f"⏩ Skipping trial {trial_number} (already completed)")
        raise optuna.TrialPruned()

    learning_rate = trial.suggest_float("learning_rate", 5e-5, 5e-4, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 4)
    lora_r = trial.suggest_categorical("lora_r", [4, 8, 16])
    lora_alpha = trial.suggest_categorical("lora_alpha", [16, 32, 64])

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )
    base_model = prepare_model_for_kbit_training(base_model)

    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )
    model = get_peft_model(base_model, lora_config)

    output_dir = os.path.join(base_output_dir, f"optuna_trial_{trial_number}")
    logging_dir = os.path.join(output_dir, "logs")

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        bf16=True,
        load_best_model_at_end=True,
        report_to="none",
        save_total_limit=1,
        logging_dir=logging_dir,
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["validation"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    print(f"🎯 Trial {trial_number} | LR: {learning_rate:.2e}, Epochs: {num_train_epochs}, R: {lora_r}, Alpha: {lora_alpha}")

    trainer.train()
    trial.set_user_attr("best_model_path", trainer.state.best_model_checkpoint or output_dir)
    eval_metrics = trainer.evaluate()
    return eval_metrics["eval_loss"]

# 📈 Resume Study
storage_path = f"sqlite:///{os.path.join(base_output_dir, 'optuna_study.db')}"
study = optuna.create_study(direction="minimize", study_name="resume_lora_ft", storage=storage_path, load_if_exists=True)

# 🔁 Continue from last completed
n_trials_total = 10
n_remaining = n_trials_total - len(completed_trials)
print(f"🔁 Running {n_remaining} more trials (out of {n_trials_total})")

#study.optimize(objective, n_trials=n_remaining)

#print("🏁 Final Best Hyperparameters:")
#print(study.best_params)


[I 2025-06-27 15:28:15,819] Using an existing study with name 'resume_lora_ft' instead of creating a new one.


✅ Detected completed trials: [0, 1, 2, 3, 4, 5, 6, 7]
🔁 Running 2 more trials (out of 10)


In [16]:
from optuna.trial import TrialState

# --- Restore completed trials into Optuna Study ---
for trial_id in completed_trials:
    trial_dir = os.path.join(base_output_dir, f"optuna_trial_{trial_id}")
    
    # Extract saved config from TrainingArguments
    trainer_state_path = None
    for subdir in os.listdir(trial_dir):
        if subdir.startswith("checkpoint-"):
            state_path = os.path.join(trial_dir, subdir, "trainer_state.json")
            if os.path.exists(state_path):
                trainer_state_path = state_path
                break

    if not trainer_state_path:
        continue

    with open(trainer_state_path) as f:
        trainer_state = json.load(f)

    # Extract saved best checkpoint path
    best_model_path = trainer_state.get("best_model_checkpoint", trial_dir)
    hparams_path = os.path.join(best_model_path, "hparams.json")
    if not os.path.exists(hparams_path):
        continue

    with open(hparams_path) as f:
        hparams = json.load(f)

    # Manually register this trial
    trial = study.ask(
        {
            "learning_rate": float(hparams.get("learning_rate", 2e-4)),
            "num_train_epochs": int(hparams.get("num_train_epochs", 3)),
            "lora_r": int(hparams.get("lora_r", 8)),
            "lora_alpha": int(hparams.get("lora_alpha", 32)),
        }
    )

    # Log final loss (fallback if missing)
    eval_loss = trainer_state.get("log_history", [{}])[-1].get("eval_loss", 2.0)
    
    # Manually tell Optuna the result
    study.tell(trial, eval_loss, state=TrialState.COMPLETE)

print(f"✅ Injected {len(completed_trials)} completed trials into Optuna study")


✅ Injected 6 completed trials into Optuna study


In [20]:
print(completed_trials)

{0, 1, 2, 3, 4, 5, 6, 7}


In [22]:
# Continue tuning only remaining trials
n_remaining = 10 - len(completed_trials)
print(f"🔁 Running {n_remaining} more trials (out of 10 total)")
study.optimize(objective, n_trials=n_remaining)

print("🏁 Final Best Hyperparameters:")
print(study.best_params)


🔁 Running 2 more trials (out of 10 total)


Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.29s/it]
/tmp/ipykernel_99891/3353892731.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🎯 Trial 10 | LR: 4.97e-04, Epochs: 3, R: 8, Alpha: 64


/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,1.688800,1.593021
2,1.390200,1.388636
3,1.332500,1.297847


...

/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,2.231900,2.159543
2,2.016700,2.052743


/home/ubuntu/AI-resume-agent/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[I 2025-06-28 09:02:02,534] Trial 11 finished with value: 2.0527429580688477 and parameters: {'learning_rate': 0.0001001872627709479, 'num_train_epochs': 2, 'lora_r': 8, 'lora_alpha': 32}. Best is trial 10 with value: 1.297846794128418.


🏁 Final Best Hyperparameters:
{'learning_rate': 0.0004966883624204826, 'num_train_epochs': 3, 'lora_r': 8, 'lora_alpha': 64}


###  Save Final Best Model 

In [23]:
best_trial = study.best_trial
best_model_path = best_trial.user_attrs.get("best_model_path")
print(f"🏆 Best model saved at: {best_model_path}")


🏆 Best model saved at: json_outputs_all_data/fine-tune/optuna_output/optuna_trial_10/checkpoint-4500


In [24]:
best_trial_number = study.best_trial.number
#best_model_path = f"optuna_output/{best_trial_number}"
best_model_path = study.best_trial.user_attrs.get("best_model_path")
print("✅ Best model path:", best_model_path)

model = AutoModelForCausalLM.from_pretrained(best_model_path)
model.save_pretrained(Config.JSON_OUTPUT_FINE_TUNE_MODEL)
tokenizer.save_pretrained(Config.JSON_OUTPUT_FINE_TUNE_MODEL)



✅ Best model path: json_outputs_all_data/fine-tune/optuna_output/optuna_trial_10/checkpoint-4500


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.40it/s]


('json_outputs_all_data/fine-tune/model/tokenizer_config.json',
 'json_outputs_all_data/fine-tune/model/special_tokens_map.json',
 'json_outputs_all_data/fine-tune/model/chat_template.jinja',
 'json_outputs_all_data/fine-tune/model/vocab.json',
 'json_outputs_all_data/fine-tune/model/merges.txt',
 'json_outputs_all_data/fine-tune/model/added_tokens.json',
 'json_outputs_all_data/fine-tune/model/tokenizer.json')

## Save to hugging face

In [25]:
from huggingface_hub import HfApi, HfFolder
from transformers import AutoTokenizer, AutoModelForCausalLM

# ✅ Optional: Login (only needed once per environment)
# from huggingface_hub import login
# login("hf_your_access_token")

# Set model path and repo name
model_path = Config.JSON_OUTPUT_FINE_TUNE_MODEL
repo_name = "rubsj/Qwen2-Resume-ATS"  # customize this

# Push model and tokenizer to HF hub
model.push_to_hub(repo_name, private=True)
tokenizer.push_to_hub(repo_name, private=True)


adapter_model.safetensors: 100%|██████████| 10.1M/10.1M [00:00<00:00, 12.5MB/s]
tokenizer.json: 100%|██████████| 11.4M/11.4M [00:00<00:00, 14.3MB/s]


CommitInfo(commit_url='https://huggingface.co/rubsj/Qwen2-Resume-ATS/commit/b2c29ebf448f72c22c07d1e322d68625de6d1d6e', commit_message='Upload tokenizer', commit_description='', oid='b2c29ebf448f72c22c07d1e322d68625de6d1d6e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rubsj/Qwen2-Resume-ATS', endpoint='https://huggingface.co', repo_type='model', repo_id='rubsj/Qwen2-Resume-ATS'), pr_revision=None, pr_num=None)

## Create ZIP for Lambda Download

In [26]:
import shutil

zip_path = f"{Config.JSON_OUTPUT_FINE_TUNE_MODEL}.zip"
shutil.make_archive(base_name=Config.JSON_OUTPUT_FINE_TUNE_MODEL, format='zip', root_dir=Config.JSON_OUTPUT_FINE_TUNE_MODEL)
print(f"✅ Model zipped at: {zip_path}")


✅ Model zipped at: json_outputs_all_data/fine-tune/model.zip


In [27]:
print(f"✅ Hugging Face Model URL: https://huggingface.co/{repo_name}")
print(f"✅ Local zip ready for download: {zip_path}")


✅ Hugging Face Model URL: https://huggingface.co/rubsj/Qwen2-Resume-ATS
✅ Local zip ready for download: json_outputs_all_data/fine-tune/model.zip


In [28]:
import shutil

zip_path = f"{Config.JSON_OUTPUT_FINE_TUNE_OUTPUT}.zip"
shutil.make_archive(base_name=Config.JSON_OUTPUT_FINE_TUNE_OUTPUT, format='zip', root_dir=Config.JSON_OUTPUT_FINE_TUNE_OUTPUT)
print(f"✅ Model zipped at: {zip_path}")


✅ Model zipped at: json_outputs_all_data/fine-tune/optuna_output.zip


In [36]:
import os

db_path = os.path.join(Config.JSON_OUTPUT_FINE_TUNE_OUTPUT, "optuna_study.db")
size_in_bytes = os.path.getsize(db_path)
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"📦 Database size: {size_in_mb:.2f} MB")


📦 Database size: 0.11 MB
